<a href="https://colab.research.google.com/github/salarbalou/steadystate_analysis/blob/main/plot_photothermal_Tvst_profile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import requests as req
import plotly.io as pio
import re
pio.templates.default ='plotly_dark'
from plotly.subplots import make_subplots

In [ ]:
#date_hint = '10-19-21'
#compound_hint = 'GNR , GNS , CQD'
#concentration_hint = '10x, 5x, 1.5x'

#please type 'date+compound+concentration'

compound_and_concentration= 'CQD_Isothermal_heating_at_95'

In [ ]:
baseurl = 'https://raw.githubusercontent.com/salarbalou/photothermal_heating/main/'
char = '.txt'
text = req.get(baseurl+compound_and_concentration+char).text
data_path = re.compile(r'^([0-9.]+)[\s,]([0-9.]+)', re.MULTILINE)
data_list = data_path.findall(text)
data = []
for tuple in data_list:
     eachtuple = []
     for xy in tuple:
          eachtuple.append(float(xy))
     data.append((eachtuple[0], eachtuple[1]))
final_data = np.array(data)
fig = make_subplots()
fig.add_scatter(x = final_data[:, 0], y = final_data[:, 1], mode = 'markers+lines', marker_color ='red' , line_color = "green")
fig.update_layout(height = 800, width = 1200,yaxis_title = 'Temperature' , xaxis_title = 'time (seconds)')